# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [101]:
# import python xml parse library
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

In [102]:
# Load and parse the xml document
xml_tree = ET.parse( './data/mondial_database.xml' )

# Confirm the root element has no attributes. Will enumerate direct children
root = xml_tree.getroot()

# list the elements that are attributes of the root 'country' tag
xml_data = root.find('country')

print(list(xml_data))


[<Element 'name' at 0x0000019A41E3ADB8>, <Element 'population' at 0x0000019A41E3AEF8>, <Element 'population' at 0x0000019A41E3AC28>, <Element 'population' at 0x0000019A41E3AD68>, <Element 'population' at 0x0000019A41E3ACC8>, <Element 'population' at 0x0000019A41E3ABD8>, <Element 'population' at 0x0000019A41E3AC78>, <Element 'population' at 0x0000019A41E3AA48>, <Element 'population' at 0x0000019A41E3AB88>, <Element 'population' at 0x0000019A41E3AB38>, <Element 'population_growth' at 0x0000019A41E3AAE8>, <Element 'infant_mortality' at 0x0000019A41E3AA98>, <Element 'gdp_total' at 0x0000019A41E3A9F8>, <Element 'gdp_agri' at 0x0000019A41E3A9A8>, <Element 'gdp_ind' at 0x0000019A41E3A958>, <Element 'gdp_serv' at 0x0000019A41E3A908>, <Element 'inflation' at 0x0000019A41E3A8B8>, <Element 'unemployment' at 0x0000019A41E3A408>, <Element 'indep_date' at 0x0000019A41E3A728>, <Element 'government' at 0x0000019A41E3A818>, <Element 'encompassed' at 0x0000019A41E3A7C8>, <Element 'ethnicgroup' at 0x0000

In [264]:
# Print tree elements to get idea of xml document structure
for elem in xml_tree.iter():
  #print (elem.tag, elem.attrib)

SyntaxError: unexpected EOF while parsing (<ipython-input-264-071ed4d692e1>, line 3)

In [113]:
# get data from xml tree
xml_data = root.find('country')
# initialize list
xml_list = []
# columns to be used in data frame
labels = ['country', 'mortality']
# initialize data frame 
df = []

# iterate through xml tree from root 'country'
for country in root.findall('country'):
    
    # find the xml 'name' element and get element text
    name = country.find('name').text
   
    # check to see if name element is blank
    if name is None:
        name = "Country not defined"
    try:    
        # find the xml 'infant_mortality' element and get element text
        mortality = country.find('infant_mortality').text
    except:
        # if 'infant_mortality' element does exist set variable to 0
        mortality = '0'
    
    # get the xml elements and add to a string variable 
    list_entry = name + ',' + mortality
   
    # separate the one string variable into separate strings
    list_entry = list_entry.split(',')
      
    # create list from xml data converted to a string
    xml_list.append(list_entry)
       
    # covert to a dataframe to better handle the data
    df = pd.DataFrame(xml_list, columns=labels)
    
# show the results of converting to a data frame; limit first 10 records 
df.head(10)
    

,country,mortality
0,Albania,13.19
1,Greece,4.78
2,Macedonia,7.9
3,Serbia,6.16
4,Montenegro,0
5,Kosovo,0
6,Andorra,3.69
7,France,3.31
8,Spain,3.33
9,Austria,4.16


#### Question 1: 10 countries with the lowest infant mortality rates

In [117]:
# Sort the data frame to get the 10 countries with the lowest mortality rates.

# filter all mortality rates = '0'
df = df[df.mortality != '0']
# show the countries with the lowest mortality rates 
df.sort_values(by='mortality', ascending=True).head(10)

,country,mortality
38,Monaco,1.81
30,Romania,10.16
153,Fiji,10.2
69,Brunei,10.48
132,Grenada,10.5
237,Mauritius,10.59
124,Panama,10.7
243,Seychelles,10.77
102,United Arab Emirates,10.92
113,Barbados,10.93


#### Question 2: 10 cities with the largest population

In [96]:
# open the xml file and parse contents to xml_tree variable
xml_tree = ET.parse( './data/mondial_database.xml' )

# initialize list variable 
xml_lst = []

# column heads for data frame
col_lbls = ['country', 'population']

# get root of xml tree 
root = xml_tree.getroot()

# interate through the city element looking for city 'name' and 'population'
for country in root.findall('country'):
        city_name = city.find('name').text
        
        try:   
            population = city.find('population').text
        except:
            population = '0'
              
              
        # get the xml elements and add to a string variable 
        city_list = city_name + ',' + population
   
        # separate the one string variable into separate strings
        city_list =  city_list.split(',')
      
        # create list from xml data converted to a string
        xml_lst.append(city_list)
       
        # covert to a dataframe to better handle the data
        ctry_df = pd.DataFrame(xml_lst, columns=col_lbls) 



In [98]:
# Sort the data frame to get the 10 cities with the largest populations 
ctry_df.sort_values(by='population', ascending=False).head(10)

,country,population
186,Cottbus,99984
555,Perm,999157
183,Erlangen,99808
2986,Laghouat,99536
2675,Alagoinhas,99508
3362,Xai-Xai,99442
2755,Camaragibe,99407
3146,Kaduna,993642
62,Nancy,99351
522,Kazan,992675


#### Question 3: 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [50]:
# open the xml file and parse contents to xml_tree variable
xml_tree = ET.parse( './data/mondial_database.xml' )

# initialize list variable 
list_entry = []

# column heads for data frame
col_lbls = ['ethnic group', 'percentage']

# get root of xml tree 
root = xml_tree.getroot()

# interate through the city element looking for city 'name' and 'population'
for country in root.findall('country'):
        try:
            e_grp = country.find('ethnicgroup').text
        except:
            e_grp = 'No ethnic group'
        
        try:   
            percentage = country.find('ethnicgroup').get('percentage')
        except:
            percentage = '0'
              
              
        # get the xml elements and add to a string variable 
        e_list = e_grp + ',' + percentage
   
        # separate the one string variable into separate strings
        e_list =  e_list.split(',')
      
        # create list from xml data converted to a string
        list_entry.append(e_list)
       
        # covert to a dataframe to better handle the data
        e_df = pd.DataFrame(list_entry, columns=col_lbls) 
e_df


,ethnic group,percentage
0,Albanian,95
1,Greek,93
2,Macedonian,64.2
3,Serb,82.9
4,Montenegrin,43
5,Albanian,92
6,Spanish,43
7,No ethnic group,0
8,Mediterranean Nordic,100
9,Austrian,91.1


In [53]:
# drop duplicates
e_drop_dups = e_df.drop_duplicates(subset=['ethnic group'],keep="first")

# Sort the data frame to get the 10 ethnic groups with the largest populations 
e_drop_dups.sort_values(by='percentage', ascending=False).head(10)


,ethnic group,percentage
233,Sotho,99.7
98,Japanese,99.4
65,Bengali,98
61,Armenian,97.7
188,Berber Arab,97
128,African,95
0,Albanian,95
204,Batswana,95
211,Bantu,95
80,Mongol,94.9


#### Question 4: name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [260]:
# open the xml file and parse contents to xml_tree variable
xml_tree = ET.parse( './data/mondial_database.xml' )

# get root of xml tree 
root = xml_tree.getroot()

col_labels = ['river', 'length', 'country']

rivers_list = []

# iterate through the river tag
for elem in xml_tree.iter(tag='river'):
     try:
         river_name = elem.find('name').text
     except:
         river_name = 'River not found'

     try: 
         river_length = elem.find('length').text
     except:
         river_length = '0'
     
     try:
        river_country = elem.get('country')
     except:
        river_country =  'Country not defined'
            
            
     # get the xml elements and add to a string variable 
     rivers_string = river_name + ',' + river_length + ',' + river_country
   
     # separate the one string variable into separate strings
     rivers_string =  rivers_string.split(',')
      
     # create list from xml data converted to a string
     rivers_list.append(rivers_string)
       
     # covert to a dataframe to better handle the data
     rivers_df = pd.DataFrame(rivers_list, columns=col_labels) 
    
     # convert the length column to numeric so it can be correctly sorted
     rivers_df[['length']] = rivers_df[['length']].apply(pd.to_numeric)

# print data frame to see structure     
rivers_df.head()

,river,length,country
0,Thjorsa,230.0,IS
1,Joekulsa a Fjoellum,206.0,IS
2,Glomma,604.0,N
3,Lagen,322.0,N
4,Goetaaelv,93.0,S


#### a. Country with the longest river 

In [263]:
# Sort the data frame to get the the longest river
rivers_df.sort_values(by='length', ascending=False).head(1)

,river,length,country
174,Amazonas,6448.0,CO BR PE


In [255]:
# open the xml file and parse contents to xml_tree variable
xml_tree = ET.parse( './data/mondial_database.xml' )

# get root of xml tree 
root = xml_tree.getroot()

col_labels = ['lake', 'area', 'country']

lakes_list = []

# iterate through the river tag
for elem in xml_tree.iter(tag='lake'):
    try:
        lake_name = elem.find('name').text
    except:
        lake_name = "Lake name not defined" 
        
    try: 
        lake_area = elem.find('area').text
    except:
        lake_area= '0'
    
    try:
        airport_country = elem.get('country')
    except:
        airport_country =  'Country not defined'
                        
    # get the xml elements and add to a string variable 
    lakes_string = lake_name + ',' + lake_area + ',' + airport_country
   
    # separate the one string variable into separate strings
    lakes_string =  lakes_string.split(',')
      
    # create list from xml data converted to a string
    lakes_list.append(lakes_string)
       
    # covert to a dataframe to better handle the data
    lakes_df = pd.DataFrame(lakes_list, columns=col_labels)      
           
    # convert the area column to numeric so it can be correctly sorted
    lakes_df[['area']] = lakes_df[['area']].apply(pd.to_numeric)

# print data frame to see structure            
lakes_df.head()

,lake,area,country
0,Inari,1040.0,SF
1,Oulujaervi,928.0,SF
2,Kallavesi,472.0,SF
3,Saimaa,4370.0,SF
4,Paeijaenne,1118.0,SF


#### b. Country with the largest lake

In [258]:
# Sort the data frame to get the the largest lakes
lakes_df.sort_values(by='area', ascending=False).head(1)

,lake,area,country
54,Caspian Sea,386400.0,R AZ KAZ IR TM


In [253]:
# open the xml file and parse contents to xml_tree variable
xml_tree = ET.parse( './data/mondial_database.xml' )

# get root of xml tree 
root = xml_tree.getroot()

col_labels = ['airport', 'elevation', 'country']

airports_string = ""

airports_list = []

# iterate through the river tag
for elem in xml_tree.iter(tag='airport'):
    try:
        airport_name = elem.find('name').text
    except:
        airport_name = "Airport name not defined"
    
    try: 
        airport_elev = elem.find('elevation').text
        if airport_elev is None:
             airport_elev =  '0'
    except:
             airport_elev =  0
    
    try:
        airport_country = elem.get('country')
                   
    except:
        airport_country =  'Country not defined'
            
    #float(airport_elev)
                        
    # get the xml elements and add to a string variable 
    airports_string = airport_name + ',' + airport_elev + ',' + airport_country
   
    # separate the one string variable into separate strings
    airports_string =  airports_string.split(',')
      
    # create list from xml data converted to a string
    airports_list.append(airports_string)
       
    # covert to a dataframe to better handle the data
    airports_df = pd.DataFrame(airports_list, columns=col_labels) 
    
    # convert the elevation column to numeric so it can be correctly sorted
    airports_df[['elevation']] = airports_df[['elevation']].apply(pd.to_numeric)

# print data frame to see structure     
airports_df.head()

,airport,elevation,country
0,Herat,977,AFG
1,Kabul Intl,1792,AFG
2,Tirana Rinas,38,AL
3,Cheikh Larbi Tebessi,811,DZ
4,Batna Airport,822,DZ


#### c. Country with the highest airport elevation 

In [252]:
# Sort the data frame to get the the highest airport elevation 
airports_df.sort_values(by='elevation', ascending=False).head(1)

,airport,elevation,country
80,El Alto Intl,4063,BOL
